In [200]:
import os
import sys 
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

sys.path.append(r'C:\Users\DerDo\Desktop\fantasy_basketball_project')
from scrape_functions import *
from database_conn import SQLiteClient

scraper = Scrape_Functions()
lebron_db_path = r"../LEBRON/lebron.db"
jordan_db_path = r"../JORDAN/jordan.db"

In [201]:
filepath = '../JORDAN/team_adv_data'
contents = os.listdir(filepath)

data = []
for file in contents:
    file_path = f'../JORDAN//team_adv_data/{file}'
    df = pd.read_csv(file_path)
    df['season'] = file.split('.')[0]
    data.append(df)

jordan_advanced_team_stats = pd.concat(data).drop(columns=['Rk'])

jordan_advanced_team_stats.columns = [col.lower() for col in jordan_advanced_team_stats]
jordan_advanced_team_stats.columns = [col.replace('2', 'two_') for col in jordan_advanced_team_stats]
jordan_advanced_team_stats.columns = [col.replace('3', 'three_') for col in jordan_advanced_team_stats]
jordan_advanced_team_stats.columns = [col.replace('%', '_perc') for col in jordan_advanced_team_stats]
jordan_advanced_team_stats.columns = [col.replace('/48', 'per48') for col in jordan_advanced_team_stats]
jordan_advanced_team_stats.columns = [col.replace('-', '_') for col in jordan_advanced_team_stats]

names = jordan_advanced_team_stats['player'].values
ages = jordan_advanced_team_stats['age'].values

filepath = '../JORDAN/team_per_pos_data'
contents = os.listdir(filepath)

data = []
for file in contents:
    file_path = f'../JORDAN//team_per_pos_data/{file}'
    df = pd.read_csv(file_path)
    df['season'] = file.split('.')[0]
    data.append(df)

jordan_per_100_possessions_team_stats = pd.concat(data).drop(columns=['Rk'])

jordan_per_100_possessions_team_stats.columns = [col.lower() for col in jordan_per_100_possessions_team_stats]
jordan_per_100_possessions_team_stats.columns = [col.replace('2', 'two_') for col in jordan_per_100_possessions_team_stats]
jordan_per_100_possessions_team_stats.columns = [col.replace('3', 'three_') for col in jordan_per_100_possessions_team_stats]
jordan_per_100_possessions_team_stats.columns = [col.replace('%', '_perc') for col in jordan_per_100_possessions_team_stats]
jordan_per_100_possessions_team_stats.columns = [col.replace('-', '_') for col in jordan_per_100_possessions_team_stats]

names = jordan_per_100_possessions_team_stats['player'].values
ages = jordan_per_100_possessions_team_stats['age'].values

filepath = '../JORDAN/game_data'
contents = os.listdir(filepath)

data = []
for file in contents:
    file_path = f'../JORDAN/game_data/{file}'
    df = pd.read_csv(file_path)
    df['season'] = file.split('.')[0]
    data.append(df)

jordan_game_data = pd.concat(data).drop(columns=['Unnamed: 5', 'Rk', 'Gcar', 'Gtm'])
jordan_game_data['GS'] = jordan_game_data['GS'].apply(lambda x: 1 if x == '*' else 0)

jordan_game_data.columns = [col.lower() for col in jordan_game_data.columns]
jordan_game_data.columns = [col.replace('2', 'two_') for col in jordan_game_data.columns]
jordan_game_data.columns = [col.replace('3', 'three_') for col in jordan_game_data.columns]
jordan_game_data.columns = [col.replace('%', '_perc') for col in jordan_game_data.columns]
jordan_game_data.columns = [col.replace('+/-', 'bpm') for col in jordan_game_data.columns]

result = jordan_game_data['result'].str.slice(0, 1)
result = [0 if r == 'L' else 1 for r in result]
scores = jordan_game_data['result'].str.split('-')

home_scores = []
away_scores = []

for score in scores:
    home_score = score[0][2:]
    away_score = score[1][0:3].strip()

    home_scores.append(home_score)
    away_scores.append(away_score)

jordan_game_data['result'] = result
jordan_game_data['team_score'] = home_scores
jordan_game_data['opp_score'] = away_scores

times = jordan_game_data['mp'].str.split(':')

seconds = []
for t in times:
    min_sec = int(t[0]) * 60
    time = min_sec + int(t[1])
    
    seconds.append(time)

jordan_game_data['mp'] = seconds

In [202]:
filepath = '../LEBRON/game_data'
contents = os.listdir(filepath)

data = []
for file in contents:
    file_path = f'../LEBRON//game_data/{file}'
    df = pd.read_csv(file_path)
    df['season'] = file.split('.')[0]
    data.append(df)

lebron_game_data = pd.concat(data).drop(columns=['Unnamed: 5', 'Rk', 'Gcar', 'Gtm'])
lebron_game_data['GS'] = lebron_game_data['GS'].apply(lambda x: 1 if x == '*' else 0)

lebron_game_data.columns = [col.lower() for col in lebron_game_data.columns]
lebron_game_data.columns = [col.replace('2', 'two_') for col in lebron_game_data.columns]
lebron_game_data.columns = [col.replace('3', 'three_') for col in lebron_game_data.columns]
lebron_game_data.columns = [col.replace('%', '_perc') for col in lebron_game_data.columns]
lebron_game_data.columns = [col.replace('+/-', 'bpm') for col in lebron_game_data.columns]

result = lebron_game_data['result'].str.slice(0, 1)
result = [0 if r == 'L' else 1 for r in result]
scores = lebron_game_data['result'].str.split('-')

home_scores = []
away_scores = []

for score in scores:
    home_score = score[0][2:]
    away_score = score[1][0:3].strip()

    home_scores.append(home_score)
    away_scores.append(away_score)

lebron_game_data['result'] = result
lebron_game_data['team_score'] = home_scores
lebron_game_data['opp_score'] = away_scores

times = lebron_game_data['mp'].str.split(':')

seconds = []
for t in times:
    min_sec = int(t[0]) * 60
    time = min_sec + int(t[1])
    
    seconds.append(time)

lebron_game_data['mp'] = seconds

filepath = '../LEBRON/team_per_pos_data'
contents = os.listdir(filepath)

data = []
for file in contents:
    file_path = f'../LEBRON//team_per_pos_data/{file}'
    df = pd.read_csv(file_path)
    df['season'] = file.split('.')[0]
    data.append(df)

lebron_per_100_possessions_team_stats = pd.concat(data).drop(columns=['Rk'])

lebron_per_100_possessions_team_stats.columns = [col.lower() for col in lebron_per_100_possessions_team_stats]
lebron_per_100_possessions_team_stats.columns = [col.replace('2', 'two_') for col in lebron_per_100_possessions_team_stats]
lebron_per_100_possessions_team_stats.columns = [col.replace('3', 'three_') for col in lebron_per_100_possessions_team_stats]
lebron_per_100_possessions_team_stats.columns = [col.replace('%', '_perc') for col in lebron_per_100_possessions_team_stats]
lebron_per_100_possessions_team_stats.columns = [col.replace('-', '_') for col in lebron_per_100_possessions_team_stats]

names = lebron_per_100_possessions_team_stats['player'].values
ages = lebron_per_100_possessions_team_stats['age'].values

filepath = '../LEBRON/team_adv_data'
contents = os.listdir(filepath)

data = []
for file in contents:
    file_path = f'../LEBRON//team_adv_data/{file}'
    df = pd.read_csv(file_path)
    df['season'] = file.split('.')[0]
    data.append(df)

lebron_advanced_team_stats = pd.concat(data).drop(columns=['Rk'])

lebron_advanced_team_stats.columns = [col.lower() for col in lebron_advanced_team_stats]
lebron_advanced_team_stats.columns = [col.replace('2', 'two_') for col in lebron_advanced_team_stats]
lebron_advanced_team_stats.columns = [col.replace('3', 'three_') for col in lebron_advanced_team_stats]
lebron_advanced_team_stats.columns = [col.replace('%', '_perc') for col in lebron_advanced_team_stats]
lebron_advanced_team_stats.columns = [col.replace('/48', 'per48') for col in lebron_advanced_team_stats]
lebron_advanced_team_stats.columns = [col.replace('-', '_') for col in lebron_advanced_team_stats]

names = lebron_advanced_team_stats['player'].values
ages = lebron_advanced_team_stats['age'].values


In [203]:
###
# For some reason not all data is present when using database 12.26
###

# with SQLiteClient(jordan_db_path) as db:
#     jordan_game_data = pd.DataFrame(
#         db.query("SELECT * FROM jordan_game_data")
#     )
#     jordan_per_100_possessions_team_stats = pd.DataFrame(
#         db.query("SELECT * FROM jordan_per_100_possessions_team_stats")
#     )
#     jordan_advanced_team_stats = pd.DataFrame(
#         db.query("SELECT * FROM jordan_advanced_team_stats")
#     )

# with SQLiteClient(lebron_db_path) as db:
#     lebron_game_data = pd.DataFrame(
#         db.query("SELECT * FROM lebron_game_data")
#     )
#     lebron_per_100_possessions_team_stats = pd.DataFrame(
#         db.query("SELECT * FROM lebron_per_100_possessions_team_stats")
#     )
#     lebron_advanced_team_stats = pd.DataFrame(
#         db.query("SELECT * FROM lebron_advanced_team_stats")
#     )

lebron_regular_game_data = lebron_game_data[
    ~lebron_game_data['season'].str.contains('playoff')
].reset_index(drop=True)
lebron_playoff_game_data = lebron_game_data[
    lebron_game_data['season'].str.contains('playoff')
].reset_index(drop=True)

jordan_regular_game_data = jordan_game_data[
    ~jordan_game_data['season'].str.contains('playoff')
].reset_index(drop=True)
jordan_playoff_game_data = jordan_game_data[
    jordan_game_data['season'].str.contains('playoff')
].reset_index(drop=True)

lebron_regular_100_possession_team = lebron_per_100_possessions_team_stats[
    ~lebron_per_100_possessions_team_stats['season'].str.contains('playoff')
].reset_index(drop=True)
lebron_playoff_100_possession_team = lebron_per_100_possessions_team_stats[
    lebron_per_100_possessions_team_stats['season'].str.contains('playoff')
].reset_index(drop=True)

jordan_regular_100_possession_team = jordan_per_100_possessions_team_stats[
    ~jordan_per_100_possessions_team_stats['season'].str.contains('playoff')
].reset_index(drop=True)
jordan_playoff_100_possession_team = jordan_per_100_possessions_team_stats[
    jordan_per_100_possessions_team_stats['season'].str.contains('playoff')
].reset_index(drop=True)

lebron_regular_advanced_team_stats = lebron_advanced_team_stats[
    ~lebron_advanced_team_stats['season'].str.contains('playoff')
].reset_index(drop=True)
lebron_playoff_advanced_team_stats = lebron_advanced_team_stats[
    lebron_advanced_team_stats['season'].str.contains('playoff')
].reset_index(drop=True)

jordan_regular_advanced_team_stats = jordan_advanced_team_stats[
    ~jordan_advanced_team_stats['season'].str.contains('playoff')
].reset_index(drop=True)
jordan_playoff_advanced_team_stats = jordan_advanced_team_stats[
    jordan_advanced_team_stats['season'].str.contains('playoff')
].reset_index(drop=True)

In [204]:
###
# Creating a player matrix of every teammate 
# Jordan had during every playoff series game
###

rows = jordan_playoff_game_data['date'].values
colz = pd.unique(
    jordan_playoff_advanced_team_stats[
        jordan_playoff_advanced_team_stats['player'] != 'Michael Jordan'
    ]['player']
)

cols = {}
for col in colz:
    cols[col] = np.zeros(len(rows))

player_matrix = pd.DataFrame(cols, index=rows).reset_index().rename(columns={'index':'date'})
player_matrix['season'] = jordan_playoff_game_data['season'].values

###
# Updating the player value to 1 if they played
# with Jordan in a game (or on his team)
###

test = jordan_playoff_advanced_team_stats[['player', 'season']]
seasons = pd.unique(test['season'])

for season in seasons:
    sample = jordan_playoff_advanced_team_stats[
        (jordan_playoff_advanced_team_stats['player'] != 'Michael Jordan') &
        (jordan_playoff_advanced_team_stats['season'] == season)
    ]

    players = sample['player'].values
        
    for i in range(len(player_matrix)):
        if player_matrix.iloc[i]['season'] == season:
            for player in players:
                player_matrix.loc[i, player] = 1

player_matrix = player_matrix.drop(columns=['season', 'date'])

jordan_playoff_game_data = pd.concat([jordan_playoff_game_data, player_matrix], axis=1)

In [205]:
###
# Creating a player matrix of every teammate 
# Jordan had during every playoff series game
###

rows = jordan_regular_game_data['date'].values
colz = pd.unique(
    jordan_regular_advanced_team_stats[
        jordan_regular_advanced_team_stats['player'] != 'Michael Jordan'
    ]['player']
)

cols = {}
for col in colz:
    cols[col] = np.zeros(len(rows))

player_matrix = pd.DataFrame(cols, index=rows).reset_index().rename(columns={'index':'date'})
player_matrix['season'] = jordan_regular_game_data['season'].values

###
# Updating the player value to 1 if they played
# with Jordan in a game (or on his team)
###

test = jordan_regular_advanced_team_stats[['player', 'season']]
seasons = pd.unique(test['season'])

for season in seasons:
    sample = jordan_regular_advanced_team_stats[
        (jordan_regular_advanced_team_stats['player'] != 'Michael Jordan') &
        (jordan_regular_advanced_team_stats['season'] == season)
    ]

    players = sample['player'].values
        
    for i in range(len(player_matrix)):
        if player_matrix.iloc[i]['season'] == season:
            for player in players:
                player_matrix.loc[i, player] = 1

player_matrix = player_matrix.drop(columns=['season', 'date'])

jordan_regular_game_data = pd.concat([jordan_regular_game_data, player_matrix], axis=1)

In [206]:
###
# Creating a player matrix of every teammate 
# Jordan had during every playoff series game
###

rows = lebron_playoff_game_data['date'].values
colz = pd.unique(
    lebron_playoff_advanced_team_stats[
        lebron_playoff_advanced_team_stats['player'] != 'LeBron James'
    ]['player']
)

cols = {}
for col in colz:
    cols[col] = np.zeros(len(rows))

player_matrix = pd.DataFrame(cols, index=rows).reset_index().rename(columns={'index':'date'})
player_matrix['season'] = lebron_playoff_game_data['season'].values

###
# Updating the player value to 1 if they played
# with Jordan in a game (or on his team)
###

test = lebron_playoff_advanced_team_stats[['player', 'season']]
seasons = pd.unique(test['season'])

for season in seasons:
    sample = lebron_playoff_advanced_team_stats[
        (lebron_playoff_advanced_team_stats['player'] != 'LeBron James') &
        (lebron_playoff_advanced_team_stats['season'] == season)
    ]

    players = sample['player'].values
        
    for i in range(len(player_matrix)):
        if player_matrix.iloc[i]['season'] == season:
            for player in players:
                player_matrix.loc[i, player] = 1

player_matrix = player_matrix.drop(columns=['season', 'date'])

lebron_playoff_game_data = pd.concat([lebron_playoff_game_data, player_matrix], axis=1)

In [207]:
###
# Creating a player matrix of every teammate 
# Jordan had during every playoff series game
###

rows = lebron_regular_game_data['date'].values
colz = pd.unique(
    lebron_regular_advanced_team_stats[
        lebron_regular_advanced_team_stats['player'] != 'LeBron James'
    ]['player']
)

cols = {}
for col in colz:
    cols[col] = np.zeros(len(rows))

player_matrix = pd.DataFrame(cols, index=rows).reset_index().rename(columns={'index':'date'})
player_matrix['season'] = lebron_regular_game_data['season'].values

###
# Updating the player value to 1 if they played
# with Jordan in a game (or on his team)
###

test = lebron_regular_advanced_team_stats[['player', 'season']]
seasons = pd.unique(test['season'])

for season in seasons:
    sample = lebron_regular_advanced_team_stats[
        (lebron_regular_advanced_team_stats['player'] != 'LeBron James') &
        (lebron_regular_advanced_team_stats['season'] == season)
    ]

    players = sample['player'].values
        
    for i in range(len(player_matrix)):
        if player_matrix.iloc[i]['season'] == season:
            for player in players:
                player_matrix.loc[i, player] = 1

player_matrix = player_matrix.drop(columns=['season', 'date'])

lebron_regular_game_data = pd.concat([lebron_regular_game_data, player_matrix], axis=1)

In [229]:
# Creating dummy variables for opponents

lebron_reg_opps = pd.get_dummies(lebron_regular_game_data['opp'], dtype=int)
lebron_reg_opps.columns = [col + '_opp' for col in lebron_reg_opps.columns]

lebron_playoff_opps = pd.get_dummies(
    lebron_playoff_game_data['opp'],
    dtype=int
)
lebron_playoff_opps.columns = [
    col + '_opp' for col in lebron_playoff_opps.columns
]


jordan_reg_opps = pd.get_dummies(jordan_regular_game_data['opp'], dtype=int)
jordan_reg_opps.columns = [col + '_opp' for col in jordan_reg_opps.columns]

jordan_playoff_opps = pd.get_dummies(
    jordan_playoff_game_data['opp'], 
    dtype=int
)
jordan_playoff_opps.columns = [
    col + '_opp' for col in jordan_playoff_opps.columns
]

# Creating dummy variables for teammates

lebron_reg_team = pd.get_dummies(lebron_regular_game_data['team'], dtype=int)
lebron_reg_team.columns = [col + '_team' for col in lebron_reg_team.columns]

lebron_playoff_team = pd.get_dummies(
    lebron_playoff_game_data['team'], dtype=int
)
lebron_playoff_team.columns = [
    col + '_team' for col in lebron_playoff_team.columns
]

jordan_reg_team = pd.get_dummies(jordan_regular_game_data['team'], dtype=int)
jordan_reg_team.columns = [col + '_team' for col in jordan_reg_team.columns]

jordan_playoff_team = pd.get_dummies(
    jordan_playoff_game_data['team'], 
    dtype=int
)
jordan_playoff_team.columns = [
    col + '_team' for col in jordan_playoff_team.columns
]

# Combining opp and team data

lebron_reg_combined =  pd.concat([lebron_reg_opps, lebron_reg_team], axis=1)
lebron_playoff_combined =  pd.concat(
    [lebron_playoff_opps, lebron_playoff_team], 
    axis=1
)

jordan_reg_combined =  pd.concat([jordan_reg_opps, jordan_reg_team], axis=1)
jordan_playoff_combined =  pd.concat(
    [jordan_playoff_opps, jordan_playoff_team], 
    axis=1
)

# Combining the opp and team data to data

lebron_regular_game_data = pd.concat(
    [lebron_regular_game_data, lebron_reg_combined], 
    axis=1
)
lebron_playoff_game_data = pd.concat(
    [lebron_playoff_game_data, lebron_playoff_combined], 
    axis=1
)

jordan_regular_game_data = pd.concat(
    [jordan_regular_game_data, jordan_reg_combined], 
    axis=1
)
jordan_playoff_game_data = pd.concat(
    [jordan_playoff_game_data, jordan_playoff_combined], 
    axis=1
)